In [1]:
# useful to autoreload the module without restarting the kernel
%load_ext autoreload
%autoreload 2

In [2]:
from mppi import InputFiles as I, Calculators as C

# Tutorial for the QeCalculator class

This tutorial describes the usage of the QeCalculator class, that manages the run of (many) calculations in
parallel with QuantumESPRESSO package.

In [32]:
run_dir = 'QeCalculator_test'

## Perform (many) scf computations for silicon

We init the PwInput object using an exsisting input file. Then we define 4 inputs with the associated names by
considering different values for the energy cutoff of the wave-functions

In [56]:
enegy_cutoffs = [40,50,60,70]

In [57]:
inputs = []
names = []

for e in enegy_cutoffs:
    inp = I.PwInput(file='IO_files/si_scf.in')
    inp.set_kpoints(points = [6,6,6])
    prefix = 'ecut_%s'%e
    inp.set_prefix(prefix)
    inp.set_energy_cutoff(e)
    inputs.append(inp)
    names.append(prefix)

In [58]:
names

['ecut_40', 'ecut_50', 'ecut_60', 'ecut_70']

Note that we have chosen the value of the prefix of the input object as the name of the file. In this way the inp, log and xml file created by QuantumESPRESSO
have the same name of the prefix folder.

Now we define an intance of the QeCalculator. For this example we use a direct scheduler, so the computations are runned in parallel using the python
multiprocessing module

In [59]:
C.QeCalculator?

Init signature: C.QeCalculator(omp='1', executable='pw.x', multiTask=True, scheduler='direct', mpi_run='mpirun -np 2', cpus_per_task=4, ntasks=3, skip=False, verbose=True)
Docstring:     
Manage (multiple) QuantumESPRESSO calculations performed in parallel. Computations
are managed by a scheduler that, in the actual implementation of the class, can
be `direct` or `slurm`.

Attributes:
   omp (:py:class:`int`) : value of the OMP_NUM_THREADS variable
   executable (:py:class:`string`) : set the executable (pw.x, ph.x, ..) of the QuantumESPRESSO package
   multiTask  (:py:class:`bool`) : specifies the usage of the calculator when called from the :class:`Dataset`.
        If multiTask is true all the runs appended to the Dataset class are passed in parallel to the calculator,
        otherwise a separate call of the :meth:`run` method is performed for each appended run
   mpi_run (:py:class:`string`) : command for the execution of mpirun, e.g. 'mpirun -np 4'
   cpus_per_task (:py:class:`in

In [60]:
code = C.QeCalculator()
code.global_options()

Initialize a parallel QuantumESPRESSO calculator with scheduler direct


{'omp': '1',
 'executable': 'pw.x',
 'multiTask': True,
 'scheduler': 'direct',
 'mpi_run': 'mpirun -np 2',
 'cpus_per_task': 4,
 'ntasks': 3,
 'skip': False,
 'verbose': True}

We run the computaion(s) passing the list with the inputs object and the associated names to the run method of the 
calculator

In [63]:
results = code.run(run_dir=run_dir,inputs=inputs,names=names,skip=True,other_variable = 1)
results

scheduler direct
Skip the computation for input ecut_40
Skip the computation for input ecut_50
Skip the computation for input ecut_60
Skip the computation for input ecut_70
Job completed


['QeCalculator_test/ecut_40.save/data-file-schema.xml',
 'QeCalculator_test/ecut_50.save/data-file-schema.xml',
 'QeCalculator_test/ecut_60.save/data-file-schema.xml',
 'QeCalculator_test/ecut_70.save/data-file-schema.xml']

In [64]:
code.run_options

{'omp': '1',
 'executable': 'pw.x',
 'multiTask': True,
 'scheduler': 'direct',
 'mpi_run': 'mpirun -np 2',
 'cpus_per_task': 4,
 'ntasks': 3,
 'skip': True,
 'verbose': True,
 'run_dir': 'QeCalculator_test',
 'inputs': [{'control': {'verbosity': "'high'",
    'pseudo_dir': "'../pseudos'",
    'calculation': "'scf'",
    'prefix': "'ecut_40'"},
   'system': {'force_symmorphic': '.true.',
    'occupations': "'fixed'",
    'ibrav': '2',
    'celldm(1)': '10.3',
    'ntyp': '1',
    'nat': '2',
    'ecutwfc': 40},
   'electrons': {'conv_thr': '1e-08'},
   'ions': {},
   'cell': {},
   'atomic_species': {'Si': ['28.086', 'Si.pbe-mt_fhi.UPF']},
   'atomic_positions': {'type': 'crystal',
    'values': [['Si', [0.125, 0.125, 0.125]],
     ['Si', [-0.125, -0.125, -0.125]]]},
   'kpoints': {'type': 'automatic', 'values': ([6, 6, 6], [0.0, 0.0, 0.0])},
   'cell_parameters': {},
   'file': 'IO_files/si_scf.in'},
  {'control': {'verbosity': "'high'",
    'pseudo_dir': "'../pseudos'",
    'calculat

We observe that, if the run of the simulation does not crash the output of the run method is a list with the the data-file-schema.xml (including their relative path) for subsequent parsing.
The elements of the list are ordered as the input objects in the inputs list. 

Instead, let see what happens if the simulation fails. For instance if we provide an empty input to code

In [41]:
inp2 = I.PwInput()

In [42]:
prefix = 'si_scf_test2'
inp2.set_prefix(prefix)
inp2

{'control': {'prefix': "'si_scf_test2'"},
 'system': {},
 'electrons': {},
 'ions': {},
 'cell': {},
 'atomic_species': {},
 'atomic_positions': {},
 'kpoints': {},
 'cell_parameters': {}}

In [43]:
result2 = code.run(inputs = [inp2], run_dir = run_dir,names=[prefix]) 
result2

scheduler direct
Executing command: cd QeCalculator_test; mpirun -np 2 pw.x -inp si_scf_test2.in > si_scf_test2.log
run0_is_running:True  
Job completed


[None]

In this case the output of the run method is None

### Usage of the skip parameter

If we repeat a calculation that has been already performed and skip = True the class skip its computation, for instance

In [44]:
results = code.run(run_dir=run_dir,inputs=inputs,names=names, skip = True)
results

scheduler direct
Skip the computation for input ecut_40
Skip the computation for input ecut_50
Skip the computation for input ecut_60
Skip the computation for input ecut_70
Job completed


['QeCalculator_test/ecut_40.save/data-file-schema.xml',
 'QeCalculator_test/ecut_50.save/data-file-schema.xml',
 'QeCalculator_test/ecut_60.save/data-file-schema.xml',
 'QeCalculator_test/ecut_70.save/data-file-schema.xml']

If we add one element to inputs and run again onlty the new element is computed

In [45]:
e = 80
inp = I.PwInput(file='IO_files/si_scf.in')
inp.set_kpoints(points = [6,6,6])
prefix = 'ecut_%s'%e
inp.set_prefix(prefix)
inp.set_energy_cutoff(e)
inputs.append(inp)
names.append(prefix)

In [46]:
results = code.run(run_dir=run_dir,inputs=inputs,names=names, skip = True)
results

scheduler direct
Skip the computation for input ecut_40
Skip the computation for input ecut_50
Skip the computation for input ecut_60
Skip the computation for input ecut_70
Executing command: cd QeCalculator_test; mpirun -np 2 pw.x -inp ecut_80.in > ecut_80.log
run0_is_running:True  
run0_is_running:True  
Job completed


['QeCalculator_test/ecut_40.save/data-file-schema.xml',
 'QeCalculator_test/ecut_50.save/data-file-schema.xml',
 'QeCalculator_test/ecut_60.save/data-file-schema.xml',
 'QeCalculator_test/ecut_70.save/data-file-schema.xml',
 'QeCalculator_test/ecut_80.save/data-file-schema.xml']

Instead if skip = False the class clean the run_dir before performing the computation, for istance

In [48]:
results = code.run(run_dir=run_dir,inputs=inputs[0:1],names=names[0:1], skip = False)
results

delete log file: QeCalculator_test/ecut_40.log
delete xml file: QeCalculator_test/ecut_40.xml
delete folder: QeCalculator_test/ecut_40.save
scheduler direct
Executing command: cd QeCalculator_test; mpirun -np 2 pw.x -inp ecut_40.in > ecut_40.log
run0_is_running:True  
Job completed


['QeCalculator_test/ecut_40.save/data-file-schema.xml']

## Perform a nscf computation for silicon. Usage of the source_dir option

We show how to perform a pw nscf calculation using the results of the first scf run as an input.

We observe that source_dir is unique, so we can run in parallel only runs that use the same directory
as source scf input.

For instance we consider two nscf computations

In [50]:
num_bands = [8,12]

In [51]:
inputs = []
names = []

for n in num_bands:
    inp = I.PwInput(file='IO_files/si_scf.in')
    inp.set_nscf(n)
    inp.set_kpoints(points = [6,6,6])
    prefix = 'bands_%s'%n
    inp.set_prefix(prefix)
    inp.set_energy_cutoff(40)
    inputs.append(inp)
    names.append(prefix)

In [52]:
names

['bands_8', 'bands_12']

In [54]:
results = code.run(inputs=inputs,run_dir=run_dir,names=names,source_dir='QeCalculator_test/ecut_40.save')
results

Copy source_dir QeCalculator_test/ecut_40.save in the QeCalculator_test/bands_8.save
Copy source_dir QeCalculator_test/ecut_40.save in the QeCalculator_test/bands_12.save
scheduler direct
Executing command: cd QeCalculator_test; mpirun -np 2 pw.x -inp bands_8.in > bands_8.log
Executing command: cd QeCalculator_test; mpirun -np 2 pw.x -inp bands_12.in > bands_12.log
run0_is_running:True  run1_is_running:True  
Job completed


['QeCalculator_test/bands_8.save/data-file-schema.xml',
 'QeCalculator_test/bands_12.save/data-file-schema.xml']

Instead, if skip = False the class delete the existing output files before running the computation again. 

In [55]:
results = code.run(inputs=inputs,run_dir=run_dir,names=names,source_dir='QeCalculator_test/ecut_40.save',skip=False)
results

delete log file: QeCalculator_test/bands_8.log
delete xml file: QeCalculator_test/bands_8.xml
delete folder: QeCalculator_test/bands_8.save
delete log file: QeCalculator_test/bands_12.log
delete xml file: QeCalculator_test/bands_12.xml
delete folder: QeCalculator_test/bands_12.save
Copy source_dir QeCalculator_test/ecut_40.save in the QeCalculator_test/bands_8.save
Copy source_dir QeCalculator_test/ecut_40.save in the QeCalculator_test/bands_12.save
scheduler direct
Executing command: cd QeCalculator_test; mpirun -np 2 pw.x -inp bands_8.in > bands_8.log
Executing command: cd QeCalculator_test; mpirun -np 2 pw.x -inp bands_12.in > bands_12.log
run0_is_running:True  run1_is_running:True  
Job completed


['QeCalculator_test/bands_8.save/data-file-schema.xml',
 'QeCalculator_test/bands_12.save/data-file-schema.xml']